In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the 'Demo' directory, two levels up
demo_path = os.path.abspath(os.path.join(current_dir, "../.."))
print(demo_path)
# Add the 'Demo' directory to the Python path
sys.path.append(demo_path)


/Users/ducdo/Repo/Demo


# Import modules


In [2]:
from langchain_community.utilities import SQLDatabase
from backend.config import CONNECTION_STRING

# Set up database connection


In [3]:
db = SQLDatabase.from_uri(CONNECTION_STRING, sample_rows_in_table_info=3)


/Users/ducdo/Repo/Demo/.venv/lib/python3.12/site-packages/langchain_community/utilities/sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(


In [15]:
print(db.dialect)
print(db.get_usable_table_names())


postgresql
['alembic_version', 'document', 'evaluation', 'item', 'langchain_pg_collection', 'langchain_pg_embedding', 'purchase_history', 'search_query', 'user']


# Get context


In [20]:
# Get context
context = db.get_context()
print(context)
print(context.keys())


{'table_info': '\nCREATE TABLE "user" (\n\tname VARCHAR NOT NULL, \n\temail VARCHAR NOT NULL, \n\tphone_number VARCHAR NOT NULL, \n\taddress VARCHAR NOT NULL, \n\tuser_id SERIAL NOT NULL, \n\tcreated_at TIMESTAMP WITHOUT TIME ZONE NOT NULL, \n\tupdated_at TIMESTAMP WITHOUT TIME ZONE NOT NULL, \n\tCONSTRAINT user_pkey PRIMARY KEY (user_id)\n)\n\n/*\n3 rows from user table:\nname\temail\tphone_number\taddress\tuser_id\tcreated_at\tupdated_at\nJohn Doe\tjohn@example.com\t1234567890\t123 Main St\t1\t2024-08-27 15:04:05.980881\t2024-08-27 15:04:05.980892\nMary Smith\tmary@example.com\t0987654321\t456 Elm St\t2\t2024-08-27 15:04:05.981068\t2024-08-27 15:04:05.981072\n*/\n\n\nCREATE TABLE alembic_version (\n\tversion_num VARCHAR(32) NOT NULL, \n\tCONSTRAINT alembic_version_pkc PRIMARY KEY (version_num)\n)\n\n/*\n3 rows from alembic_version table:\nversion_num\nb72fd792d675\n*/\n\n\nCREATE TABLE document (\n\tsource VARCHAR NOT NULL, \n\ttitle VARCHAR NOT NULL, \n\tdescription VARCHAR, \n\tlan

In [19]:
print(context["table_info"])


CREATE TABLE "user" (
	name VARCHAR NOT NULL, 
	email VARCHAR NOT NULL, 
	phone_number VARCHAR NOT NULL, 
	address VARCHAR NOT NULL, 
	user_id SERIAL NOT NULL, 
	created_at TIMESTAMP WITHOUT TIME ZONE NOT NULL, 
	updated_at TIMESTAMP WITHOUT TIME ZONE NOT NULL, 
	CONSTRAINT user_pkey PRIMARY KEY (user_id)
)

/*
3 rows from user table:
name	email	phone_number	address	user_id	created_at	updated_at
John Doe	john@example.com	1234567890	123 Main St	1	2024-08-27 15:04:05.980881	2024-08-27 15:04:05.980892
Mary Smith	mary@example.com	0987654321	456 Elm St	2	2024-08-27 15:04:05.981068	2024-08-27 15:04:05.981072
*/


CREATE TABLE alembic_version (
	version_num VARCHAR(32) NOT NULL, 
	CONSTRAINT alembic_version_pkc PRIMARY KEY (version_num)
)

/*
3 rows from alembic_version table:
version_num
b72fd792d675
*/


CREATE TABLE document (
	source VARCHAR NOT NULL, 
	title VARCHAR NOT NULL, 
	description VARCHAR, 
	language VARCHAR NOT NULL, 
	page_content VARCHAR NOT NULL, 
	id SERIAL NOT NULL, 
	CON

In [21]:
print(context["table_names"])


alembic_version, document, evaluation, item, langchain_pg_collection, langchain_pg_embedding, purchase_history, search_query, user


# Chain


## Convert question to sql query


In [29]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "What things that Jon bought"})
response


'SELECT i.item_name, i.price, i.brand, i.color, i.size\nFROM item i\nJOIN purchase_history ph ON i.item_id = ph.item_id\nJOIN "user" u ON ph.user_id = u.user_id\nWHERE u.name = \'John Doe\'\nLIMIT 5;'

In [30]:
print(response)

SELECT i.item_name, i.price, i.brand, i.color, i.size
FROM item i
JOIN purchase_history ph ON i.item_id = ph.item_id
JOIN "user" u ON ph.user_id = u.user_id
WHERE u.name = 'John Doe'
LIMIT 5;


In [31]:
db.run(response)


"[('Running Shoe', 89.99, 'Nike', 'Blue', '10'), ('Mechanical Pencil', 5.99, 'Pentel', 'Black', '0.5mm'), ('Travel Backpack', 89.99, 'Osprey', 'Black', '30L')]"